In [1]:
import torch
from datasets import load_dataset
from datasets import load_from_disk

# from d2l import torch as d2l


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# 定义数据集
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, split):
        # dataset = load_dataset('lansinuote/ChnSentiCorp', keep_in_memory=True)
        dataset = load_from_disk('./data/ChnSentiCorp')
        
        print(dataset)
        def f(data):
            return len(data['text']) > 30
        self.dataset = dataset.filter(f)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset[i]['text']
        return text

In [3]:
dataset = Dataset('train')
len(dataset) # , dataset[0]

Loading cached processed dataset at /home/mylady/code/python/DL-pytorch/apps/huggingface/data/ChnSentiCorp/validation/cache-cf45964edee402a8.arrow
Loading cached processed dataset at /home/mylady/code/python/DL-pytorch/apps/huggingface/data/ChnSentiCorp/test/cache-c6f7400aef16ddba.arrow
Loading cached processed dataset at /home/mylady/code/python/DL-pytorch/apps/huggingface/data/ChnSentiCorp/train/cache-478819d08c52879a.arrow


DatasetDict({
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9600
    })
})


3

In [4]:
from transformers import BertTokenizer
from transformers import BertModel


# 加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-chinese')

print(token)


# 加载预训练模型
bert = BertModel.from_pretrained('bert-base-chinese').to(device)

print('bert 预训练模型加载完毕..')

BertTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


bert 预训练模型加载完毕..


In [ ]:

def write_data():
    count = 0

    # 写入到文件
    with open('token_word.txt', 'w', encoding='utf-8') as f:
        for item in token.get_vocab():
            f.write(item)
            count += 1
    pass

In [5]:
def collate_fn(data):
    
    # print(data)
    # print('data长度: ', len(data))
    # 编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=data,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=30,
                                   return_tensors='pt',
                                   return_length=True)

    # print('编码后的data打印: ', data.keys())
    
    # input_ids:编码之后的数字
    # attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']

    # 把第15个词固定替换为mask
    # print('第15个词: %s' % input_ids[:, 15])
    
    # 这里直接使用了编码后的数据作为真实预测值
    labels = input_ids[:, 15].reshape(-1).clone()
    input_ids[:, 15] = token.get_vocab()[token.mask_token]

    # print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels

## 数据加载器

In [6]:
# 数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset['train'],
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True
                                    )


for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
    print(len(loader))
    print(token.decode(input_ids[0]))
    print(token.decode(labels[0]))
    print(input_ids.shape, attention_mask.shape, token_type_ids.shape, labels.shape)
    print("")
    
    if i >= 2:
        break

574
[CLS] 价 廉 物 美 价 廉 物 美 价 廉 物 美 价 廉 [MASK] 美 价 廉 物 美 价 廉 物 美 价 廉 物 美 [SEP]
物
torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16])

574
[CLS] 虽 然 很 喜 欢 这 本 书 ， 但 是 还 是 有 [MASK] 小 失 望 的 。 。 。 不 知 道 是 不 是 [SEP]
点
torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16])

574
[CLS] 价 格 要 是 能 再 低 点 就 更 好 了 ！ 京 [MASK] 应 该 多 搞 特 价 活 动! 快 递 的 服 [SEP]
东
torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16])



In [38]:
len(loader)

574

## 加载预训练模型

In [7]:

# 不训练,不需要计算梯度
for param in bert.parameters():
    param.requires_grad_(False)


def test_1():
    # 模型试算
    out = bert(input_ids=input_ids,
               attention_mask=attention_mask,
               token_type_ids=token_type_ids
               )

    out.last_hidden_state.shape

## 定义下游任务模型

In [8]:

# 定义下游任务模型
class Model(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.decoder = torch.nn.Linear(768, token.vocab_size, bias=False)
        self.decoder.bias = torch.nn.Parameter(torch.zeros(token.vocab_size))

    def forward(self, input_ids, attention_mask, token_type_ids):
        # pretrained = pretrained.to(device)
        with torch.no_grad():
            out = bert(input_ids=input_ids,
                      attention_mask=attention_mask,
                      token_type_ids=token_type_ids)
            pass
        
        out = self.decoder(out.last_hidden_state[:, 15])
        return out


model = Model()


#model(input_ids=input_ids,
#      attention_mask=attention_mask,
#      token_type_ids=token_type_ids
#     ).shape


# 模型转移到GPU上
model.to(device)

Model(
  (decoder): Linear(in_features=768, out_features=21128, bias=True)
)

In [9]:
list(model.parameters())[0].device

device(type='cuda', index=0)

## 训练

In [10]:
from transformers import AdamW


# 训练
optimizer = AdamW(model.parameters(), lr=5e-4)

# 损失函数
loss = torch.nn.CrossEntropyLoss()

model.train()


print("training on ", device)

training on  cuda


/home/mylady/.virtualenvs/dl-pytorch/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [11]:

def train_full_int_blank():
    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
        # print('labels', labels)
        # labels tensor([2523, 1962, ....,  6817, 1962])

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        labels = labels.to(device)

        out = model(input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids
                   )
        
        l = loss(out, labels)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()

        if i % 200 == 0:
            out = out.cpu()
            labels = labels.cpu()
            out = out.argmax(dim=1)
            accuracy = (out == labels).sum().item() / len(labels)

            print('训练批次: %s 当前第 %s 轮训练 loss: %s acc: %s' % \
                  (epoch, i, l.item(), accuracy))

In [12]:

epoch = 5

for epoch in range(epoch):
    train_full_int_blank()
    pass

训练批次: 0 当前第 0 轮训练 loss: 10.15749740600586 acc: 0.0
训练批次: 0 当前第 200 轮训练 loss: 5.76514196395874 acc: 0.1875
训练批次: 0 当前第 400 轮训练 loss: 4.217146873474121 acc: 0.4375
训练批次: 1 当前第 0 轮训练 loss: 1.9062347412109375 acc: 0.6875
训练批次: 1 当前第 200 轮训练 loss: 1.8172128200531006 acc: 0.5625
训练批次: 1 当前第 400 轮训练 loss: 1.8344106674194336 acc: 0.6875
训练批次: 2 当前第 0 轮训练 loss: 1.0104882717132568 acc: 0.875
训练批次: 2 当前第 200 轮训练 loss: 1.824883222579956 acc: 0.75
训练批次: 2 当前第 400 轮训练 loss: 0.5277410745620728 acc: 0.9375
训练批次: 3 当前第 0 轮训练 loss: 0.5519086718559265 acc: 0.9375
训练批次: 3 当前第 200 轮训练 loss: 0.6763061881065369 acc: 0.8125
训练批次: 3 当前第 400 轮训练 loss: 0.5538439750671387 acc: 0.875
训练批次: 4 当前第 0 轮训练 loss: 0.35573217272758484 acc: 0.9375
训练批次: 4 当前第 200 轮训练 loss: 0.34219175577163696 acc: 0.9375
训练批次: 4 当前第 400 轮训练 loss: 0.3749438226222992 acc: 1.0


## 模型保存

In [ ]:
# 保存

model_save_path = 'chinese_full_vacant_mission_2023_4_10.pt'
# torch.save(model.state_dict(),  model_save_path)  # 推荐的文件后缀名是pt或pth


In [ ]:

# 加载
# 加载保存的模型
model = Model()
model.load_state_dict(torch.load(model_save_path))

In [40]:
# 加载测试数据集
loader_test = torch.utils.data.DataLoader(dataset=dataset['train'],
                                          batch_size=32,
                                          collate_fn=collate_fn,
                                          shuffle=True,
                                          drop_last=True)

In [41]:
#测试
def test_calculate(stop_num, loader_test=loader_test):
    
    model.eval()
    correct = 0
    total = 0
    
    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader_test):
        if i >= stop_num:
            break
        with torch.no_grad():
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            labels = labels.to(device)
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)
            pass
    
        out = out.cpu()
        labels = labels.cpu()
        out = out.argmax(dim=1)
        accuracy = (out == labels).sum().item() / len(labels)
        
        correct += accuracy
        total += 1
        
        y = token.decode(labels[0])
        y_hat = token.decode(out[0])
        
        if y != y_hat:
            print('序: %s 输入内容: %s' % (i, token.decode(input_ids[0])))
            print('[ERROR] label: %s, \t y_hat: %s' %(y, y_hat))
            print("")
        else:
            pass

    print('total: %s acc: %.2f' % (total, correct / total))
    pass

In [42]:
# 预测
test_calculate(200)

序: 31 输入内容: [CLS] 我 不 是 个 偏 激 的 人 ， 所 以 我 喜 欢 [MASK] 各 种 各 样 的 书 。 韩 寒 和 郭 敬 明 [SEP]
[ERROR] label: 都, 	 y_hat: 看

序: 61 输入内容: [CLS] 《 不 一 样 的 卡 梅 拉 》 的 确 不 一 样 [MASK] 家 里 那 么 多 图 书 ， 儿 子 偏 偏 对 [SEP]
[ERROR] label: ！, 	 y_hat: ，

序: 75 输入内容: [CLS] 驱 动 盘 只 有 [UNK] 的 驱 动 ， [UNK] 装 成 [UNK] [MASK] 相 信 99 % 的 人 不 会 用 。 笔 记 本 [SEP]
[ERROR] label: 我, 	 y_hat: ，

序: 82 输入内容: [CLS] 镜 面 设 计 容 易 留 指 纹 ， 降 价 速 度 [MASK] 快 ， 从 订 货 到 收 货 100 元 没 了 ， [SEP]
[ERROR] label: 太, 	 y_hat: 很

序: 127 输入内容: [CLS] 入 住 这 个 酒 店 ， 应 该 算 是 被 携 程 [MASK] 误 导 了 ， 把 它 放 在 了 第 一 选 择 [SEP]
[ERROR] label: 给, 	 y_hat: 的

序: 146 输入内容: [CLS] 见 这 么 多 人 买 ， 也 买 了 一 本 感 觉 [MASK] 内 容 十 分 空 洞 ， 实 质 性 东 西 不 [SEP]
[ERROR] label: ，, 	 y_hat: 是

序: 147 输入内容: [CLS] cpu 的 使 用 较 大 ， 不 知 道 寿 命 如 何 [MASK] 显 示 屏 中 间 的 华 硕 标 志 非 常 热 [SEP]
[ERROR] label: ？, 	 y_hat: ，

序: 150 输入内容: [CLS] 使 用 效 果 很 好 ， 噪 音 不 大 ， 手 托 [MASK] 不 热 ， 屏 幕 是 2008 年 的 三 星 屏 ， [SEP]
[ERROR] label: 处, 	 y_hat: 也

序: 159 输入内容: [CLS] 总 体 来 说 还 不 错 ， 完 美 屏 ， 配 置 [MASK] 好 